In [1]:
import pandas as pd

In [6]:
dataset = pd.read_excel('data fsds/Superstore_USA.xlsx')
returns = pd.read_excel('data fsds/Superstore_USA.xlsx',sheet_name=1)
users = pd.read_excel('data fsds/Superstore_USA.xlsx',sheet_name=2)
df = dataset.copy()

In [66]:
df.head(1)

,Row ID,Order Priority,Discount,Unit Price,Shipping Cost,Customer ID,Customer Name,Ship Mode,Customer Segment,Product Category,...,Region,State or Province,City,Postal Code,Order Date,Ship Date,Profit,Quantity ordered new,Sales,Order ID
0,18606,Not Specified,0.01,2.88,0.5,2,Janice Fletcher,Regular Air,Corporate,Office Supplies,...,Central,Illinois,Addison,60101,2012-05-28,2012-05-30,1.32,2,5.9,88525


In [67]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9426 entries, 0 to 9425
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Row ID                9426 non-null   int64         
 1   Order Priority        9426 non-null   object        
 2   Discount              9426 non-null   float64       
 3   Unit Price            9426 non-null   float64       
 4   Shipping Cost         9426 non-null   float64       
 5   Customer ID           9426 non-null   int64         
 6   Customer Name         9426 non-null   object        
 7   Ship Mode             9426 non-null   object        
 8   Customer Segment      9426 non-null   object        
 9   Product Category      9426 non-null   object        
 10  Product Sub-Category  9426 non-null   object        
 11  Product Container     9426 non-null   object        
 12  Product Name          9426 non-null   object        
 13  Product Base Margi

In [148]:
returns.shape

(1634, 2)

In [10]:
users.head()

,Region,Manager
0,Central,Chris
1,East,Erin
2,South,Sam
3,West,William


In [4]:
df.columns

Index(['Row ID', 'Order Priority', 'Discount', 'Unit Price', 'Shipping Cost',
       'Customer ID', 'Customer Name', 'Ship Mode', 'Customer Segment',
       'Product Category', 'Product Sub-Category', 'Product Container',
       'Product Name', 'Product Base Margin', 'Region', 'State or Province',
       'City', 'Postal Code', 'Order Date', 'Ship Date', 'Profit',
       'Quantity ordered new', 'Sales', 'Order ID'],
      dtype='object')

# Find out how return that we ahve recived and with a product id

In [28]:
df.merge(returns,on='Order ID',how='inner')[['Row ID','Status']]

,Row ID,Status
0,1359,Returned
1,1950,Returned
2,1951,Returned
3,5055,Returned
4,5573,Returned
...,...,...
93,986,Returned
94,987,Returned
95,6041,Returned
96,1910,Returned


# try  to join order and return data both in sql and pandas 

In [152]:
df = df.merge(returns,how='outer')

# Try to find out how many unique customer that we have 

In [134]:
df['Customer ID'].nunique()

2703

# try to find out in how many regions we are selling a product and who is a manager for a respective region

In [154]:
df['Region'].unique()
# There are Four Regions and following are the respective managers
df = df.merge(users,on='Region')
df.groupby('Region')['Manager'].unique()

Region
Central      [Chris]
East          [Erin]
South          [Sam]
West       [William]
Name: Manager, dtype: object

# find out how many different differnet shipement mode that we have and what is a percentage usablity of all the shipment mode with respect to dataset

In [155]:
print(df['Ship Mode'].nunique())
print(df['Ship Mode'].unique())

3
['Regular Air' 'Express Air' 'Delivery Truck']


In [156]:
percent_ship_mode = pd.DataFrame(df.groupby('Ship Mode')['Order ID'].count()/len(df) * 100)
percent_ship_mode.columns = ['Percent_usability']
percent_ship_mode

,Percent_usability
Ship Mode,
Delivery Truck,13.611288
Express Air,11.744112
Regular Air,74.644600


# Create a new column and try to find our a diffrence between order date and shipment date

In [157]:
df['Shipment_duration'] = df['Order Date'] - df['Ship Date']
df['Shipment_duration'].head()

0   -2 days
1   -2 days
2   -1 days
3   -1 days
4   -1 days
Name: Shipment_duration, dtype: timedelta64[ns]

# based on question number 8 find out for which order id we have shipment duration more than 10 days 

In [158]:
df[df['Shipment_duration'] < pd.Timedelta('- 10 days')][['Order ID','Shipment_duration']]

,Order ID,Shipment_duration
189,87215,-84 days
459,86318,-11 days
460,86318,-15 days
1726,88223,-14 days
3476,87957,-17 days
3477,87957,-11 days
3478,87957,-28 days
3495,19556,-17 days
3496,19556,-11 days
3497,19556,-28 days


# Try to find out a list of a returned order which sihpment duration was more then 15 days and find out that region manager as well

In [163]:
df[(df['Status'] == 'Returned') & (df['Shipment_duration'] < pd.Timedelta('-5 days'))][['Order ID','Shipment_duration','Manager']]

,Order ID,Shipment_duration,Manager
3814,51559,-9 days,William
4873,32931,-7 days,William
6054,41760,-7 days,Erin


# Group by region and find out which region is more profitable

In [176]:
df.groupby('Region')['Profit'].sum().sort_values(ascending =False)

Region
Central    519825.567067
East       377566.186045
West       310849.453897
South      104201.192420
Name: Profit, dtype: float64

In [179]:
# Most profitable 
df.groupby('Region')['Profit'].sum().sort_values(ascending =False).head(1)

Region
Central    519825.567067
Name: Profit, dtype: float64

# Try to find out overal in which country we are giving more discount 

In [196]:
df.groupby('State or Province')['Discount'].max().sort_values(ascending =False).head(1)

State or Province
Illinois    0.25
Name: Discount, dtype: float64

In [202]:
# Alternate method
df[df['Discount'] == df['Discount'].max()]

,Row ID,Order Priority,Discount,Unit Price,Shipping Cost,Customer ID,Customer Name,Ship Mode,Customer Segment,Product Category,...,Postal Code,Order Date,Ship Date,Profit,Quantity ordered new,Sales,Order ID,Shipment_duration,Status,Manager
2740,18176.0,Critical,0.25,71.37,69.0,3131.0,Jonathan Reilly,Regular Air,Home Office,Furniture,...,60056.0,2013-02-14,2013-02-16,-481.041,3.0,181.03,86802,-2 days,NaN,Chris


# Give me a list of unique postal code 

In [205]:
df['Postal Code'].unique()

array([60101., 55372., 68005., ..., 30144., 30240., 31204.])

# which customer segement is more profitalble find it out

In [217]:
df.groupby('Customer Segment')['Profit'].sum().sort_values(ascending = False)

Customer Segment
Corporate         505538.627783
Small Business    316474.592482
Home Office       283869.553814
Consumer          206559.625348
Name: Profit, dtype: float64

In [218]:
df.groupby('Customer Segment')['Profit'].sum().sort_values(ascending = False).head(1)

Customer Segment
Corporate    505538.627783
Name: Profit, dtype: float64

# try to find out the 10th most loss making product catagory

In [238]:
pd.DataFrame(df.groupby('Product Name')['Profit'].sum().sort_values(ascending = True)).iloc[9]

Profit   -9543.6525
Name: Eldon ClusterMat Chair Mat with Cordless Antistatic Protection, dtype: float64

# Try to find out 10 top  product with highest margins 

In [256]:
df.groupby('Product Name')['Product Base Margin'].mean().sort_values(ascending = False).head(10)

Product Name
Sterling Rubber Bands by Alliance                                             0.85
Belkin OmniView SE Rackmount Kit                                              0.85
Office Impressions Heavy Duty Welded Shelving & Multimedia Storage Drawers    0.85
Accessory27                                                                   0.85
Accessory4                                                                    0.85
Tennsco Commercial Shelving                                                   0.84
Serrated Blade or Curved Handle Hand Letter Openers                           0.84
Tennsco Stur-D-Stor Boltless Shelving, 5 Shelves, 24" Deep, Sand              0.84
Accessory36                                                                   0.83
Fellowes Officeware™ Wire Shelving                                            0.83
Name: Product Base Margin, dtype: float64